## 1. train textCNN model
## 2. extract the middle layer Vector as document distribution presentation
## 3. dimension-reduction 
## 4. data visualization

In [1]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
!pip install pandas

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
!pip install bs4

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
!pip install sklearn

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [6]:
# from keras.models import Sequential
# from keras.layers import Conv1D, GlobalMaxPooling1D, Embedding, LSTM, MaxPooling1D,BatchNormalization,GRU, SpatialDropout1D
# from keras.layers.core import Dense, Dropout, Activation
# from keras.preprocessing.text import Tokenizer
# from keras import metrics, regularizers
# from keras.preprocessing import sequence
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.models import Sequential
# from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
# from sklearn.model_selection import train_test_split
# from keras.utils.np_utils import to_categorical
# from keras.callbacks import EarlyStopping
# from keras.layers import Dropout
from bs4 import BeautifulSoup
from tensorflow.keras.callbacks import EarlyStopping
import re
# from nltk.corpus import stopwords
# import nltk
# from nltk import word_tokenize
# STOPWORDS = set(stopwords.words('english'))
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

# import Job Description dataset

In [7]:
data = pd.read_csv(r'../Data/JD.csv',usecols = ['Query','Description'])
data

,Query,Description
0,Administrative Assistant,This Administrative Assistant position is resp...
1,Administrative Assistant,ADMINISTRATIVE ASSISTANT Part Time The West Or...
2,Administrative Assistant,Administrative Assistant - In Bus 26yrs Fashi...
3,Sales Representative,Are you ready for something new? Are you tired...
4,Customer Service Representative,Superior Staff Resources is currently seeking ...
...,...,...
72287,Business Analyst,ability construct complex sql statement analyz...
72288,Business Analyst,join u pursue disruptive new vision make machi...
72289,Business Analyst,duty high level independent decision making au...
72290,Business Analyst,crowdstrike leading provider next-generation e...


In [8]:
data.Query.value_counts()

Administrative Assistant                                      4395
Customer Service Representative                               4200
Own Your Own Franchise!                                       3701
Sales Representative                                          3556
Mobile Tool Sales / Franchise Distributor                     3275
Retail Wireless Sales Consultant                              3194
Sales / Franchise                                             3120
Project Manager                                               2845
Staff Accountant                                              2834
Retail Sales Associate                                        2769
Business Analyst                                              2420
Sales / Customer Service – Part or Full time – Summer Work    2419
Sales Representative / Account Manager /  Customer Service    2412
Senior Accountant                                             2216
Jani-King Franchise Business Opportunity                      

# data clean

In [9]:
data = data.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

def clean_text(text):
#     text = BeautifulSoup(text, "lxml").text
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
#     pat = re.compile('<[^>]+>', re.S)
    text = cleanr.sub(' ', text)
    
    text = text.replace('\\n', ' ').replace('\\r', ' ').replace('\r', ' ').replace('\n', ' ')
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
#     text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split()) # remove stopwors from text
    return text

In [10]:
data['Description'] = data['Description'].apply(clean_text)
data['Query'] = data['Query'].apply(clean_text)
data

,Query,Description
0,administrative assistant,this administrative assistant position is resp...
1,administrative assistant,administrative assistant part time the west or...
2,administrative assistant,administrative assistant in bus 26yrs fashion ...
3,sales representative,are you ready for something new are you tired ...
4,customer service representative,superior staff resources is currently seeking ...
...,...,...
72287,business analyst,ability construct complex sql statement analyz...
72288,business analyst,join u pursue disruptive new vision make machi...
72289,business analyst,duty high level independent decision making au...
72290,business analyst,crowdstrike leading provider nextgeneration en...


# sample

In [11]:
data['Description'][0]

'this administrative assistant position is responsible for performing a variety of clerical and administrative support functions in the areas of communications data entry and retrieval typing filing copying and coordination of related administrative activities job requirements attention to detail ability to work in a fast paced environment invoicing proficient in word excel and outlook other administrative responsibilities as needed qualifications high school diploma administrative assistant experience 12 years work experience working knowledge of all basic office machines fax copier 10key etc strong verbal and written communication skills strong work ethic'

In [12]:
import re
from bs4 import BeautifulSoup

def string_process(l):
#     temp_list = [str(i).replace('\\n', ' ').replace('\\r', ' ').replace('\r', ' ').replace('\n', ' ').strip().lower() for i in nlp(l)]
#     temp_string = ' '.join( temp_list )
#     temp_list = [str(i) for i in nlp(temp_string)] 
    temp_list = l.split()[:500-2]
    return temp_list

In [13]:
data['text_clean'] = data['Description'].apply( string_process )
data['len'] = data['text_clean'].apply(len)
data

,Query,Description,text_clean,len
0,administrative assistant,this administrative assistant position is resp...,"[this, administrative, assistant, position, is...",90
1,administrative assistant,administrative assistant part time the west or...,"[administrative, assistant, part, time, the, w...",70
2,administrative assistant,administrative assistant in bus 26yrs fashion ...,"[administrative, assistant, in, bus, 26yrs, fa...",308
3,sales representative,are you ready for something new are you tired ...,"[are, you, ready, for, something, new, are, yo...",83
4,customer service representative,superior staff resources is currently seeking ...,"[superior, staff, resources, is, currently, se...",254
...,...,...,...,...
72287,business analyst,ability construct complex sql statement analyz...,"[ability, construct, complex, sql, statement, ...",148
72288,business analyst,join u pursue disruptive new vision make machi...,"[join, u, pursue, disruptive, new, vision, mak...",265
72289,business analyst,duty high level independent decision making au...,"[duty, high, level, independent, decision, mak...",77
72290,business analyst,crowdstrike leading provider nextgeneration en...,"[crowdstrike, leading, provider, nextgeneratio...",315


In [14]:
!pip install keras_bert

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [15]:
from keras_bert import load_trained_model_from_checkpoint

In [16]:
vocab = {}
vocab_path = r"uncased_L-2_H-128_A-2/vocab.txt"

with open(vocab_path, 'r+', encoding='utf-8') as f_vocab:
    for line in f_vocab.readlines():
        vocab[line.strip()] = len(vocab)

In [17]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

XX      = [ [vocab.get(str.lower(w),1) for w in l]for l in data['text_clean']]
XX      = [ [101] + l + [102] for l in XX]

XX_mask = [ [1 for w in l]for l in XX]

######################################################################################
XX      = pad_sequences(maxlen = 512, sequences = XX, padding = 'post', value = 0)
XX_mask = pad_sequences(maxlen = 512, sequences = XX_mask, padding = 'post', value = 0)

######################################################################################
print( XX[0] )
print()
print( XX_mask[0] )

[  101  2023  3831  3353  2597  2003  3625  2005  4488  1037  3528  1997
 23106  1998  3831  2490  4972  1999  1996  2752  1997  4806  2951  4443
  1998 26384 22868 15242 24731  1998 12016  1997  3141  3831  3450  3105
  5918  3086  2000  6987  3754  2000  2147  1999  1037  3435 13823  4044
     1 27029  1999  2773 24970  1998 17680  2060  3831 10198  2004  2734
 15644  2152  2082  9827  3831  3353  3325  2260  2086  2147  3325  2551
  3716  1997  2035  3937  2436  6681     1     1     1  4385  2844 12064
  1998  2517  4807  4813  2844  2147     1   102     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [18]:
YY = pd.get_dummies(data['Query']).values
print( YY[0] )

[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [19]:
max_seq_length = 512

In [20]:
def get_argmax( l ):
    return np.argmax(l)

In [21]:
data['category'] = list( YY )
data['category'] = data['category'].apply( get_argmax )
data

,Query,Description,text_clean,len,category
0,administrative assistant,this administrative assistant position is resp...,"[this, administrative, assistant, position, is...",90,2
1,administrative assistant,administrative assistant part time the west or...,"[administrative, assistant, part, time, the, w...",70,2
2,administrative assistant,administrative assistant in bus 26yrs fashion ...,"[administrative, assistant, in, bus, 26yrs, fa...",308,2
3,sales representative,are you ready for something new are you tired ...,"[are, you, ready, for, something, new, are, yo...",83,25
4,customer service representative,superior staff resources is currently seeking ...,"[superior, staff, resources, is, currently, se...",254,6
...,...,...,...,...,...
72287,business analyst,ability construct complex sql statement analyz...,"[ability, construct, complex, sql, statement, ...",148,4
72288,business analyst,join u pursue disruptive new vision make machi...,"[join, u, pursue, disruptive, new, vision, mak...",265,4
72289,business analyst,duty high level independent decision making au...,"[duty, high, level, independent, decision, mak...",77,4
72290,business analyst,crowdstrike leading provider nextgeneration en...,"[crowdstrike, leading, provider, nextgeneratio...",315,4


In [22]:
temp_DF = data[['Query','category']].drop_duplicates()

category_dict = {}
for ind, row in temp_DF.iterrows():
    category_dict[ row['category'] ] = row['Query']

category_dict

{2: 'administrative assistant',
 25: 'sales representative',
 6: 'customer service representative',
 12: 'java developer',
 9: 'financial analyst',
 17: 'project manager',
 8: 'executive assistant',
 13: 'maintenance technician',
 16: 'physical therapist',
 29: 'store manager',
 28: 'staff accountant',
 10: 'inside sales representative',
 0: 'account executive',
 18: 'receptionist',
 27: 'senior accountant',
 4: 'business analyst',
 19: 'restaurant manager',
 1: 'account representative',
 20: 'retail sales associate',
 26: 'sales representative account manager customer service',
 21: 'retail wireless sales consultant',
 24: 'sales management trainee',
 11: 'janiking franchise business opportunity',
 15: 'own your own franchise',
 5: 'cashier',
 23: 'sales franchise',
 7: 'customer service sales representative part time work',
 14: 'mobile tool sales franchise distributor',
 3: 'benefits consultant',
 22: 'sales customer service part or full time summer work'}

In [23]:
# X_train, X_test, Y_train, Y_test = train_test_split( XX,YY, test_size = 0.2, random_state = 42)
# print( X_train.shape, Y_train.shape )
# print( X_test.shape, Y_test.shape )

In [24]:
YY[0]

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [25]:
YY.shape

(72292, 30)

# Bert  Embeding

In [26]:
from tensorflow import keras


In [27]:
max_seq_length = 512
feature_size = max_seq_length
dropout_rate = 0.2
num_classes  = len( set( data['Query'] ) )


model_path = r"uncased_L-2_H-128_A-2/"
bert = load_trained_model_from_checkpoint(
    model_path + "bert_config.json",
    model_path + "bert_model.ckpt",
    seq_len = max_seq_length
)

for layer in bert.layers:
    layer.trainable = True
    
    
x1 = keras.Input(shape=(None,))
x2 = keras.Input(shape=(None,))
embed = bert([x1, x2])

# embed = keras.layers.Reshape((512, 128, 1), name='add_channel')(embed)

outputs =  tf.keras.layers.Lambda(lambda x: x[:, 0])(embed) 
# pool_outputs = []
# for filter_size in [3,4,5]:
#     filter_shape = (filter_size, 50)
#     conv = keras.layers.Conv2D(50, filter_shape, strides=(1, 1), padding='valid',
#                                data_format='channels_last', activation='relu',
#                                kernel_initializer='glorot_normal',
#                                bias_initializer=keras.initializers.constant(0.1),
#                                name='convolution_{:d}'.format(filter_size))(embed)
    
#     max_pool_shape = (feature_size - filter_size + 1, 1)
#     pool = keras.layers.MaxPool2D(pool_size=max_pool_shape,
#                                   strides=(1, 1), padding='valid',
#                                   data_format='channels_last',
#                                   name='max_pooling_{:d}'.format(filter_size))(conv)
    
#     pool_outputs.append(pool)

# pool_outputs = keras.layers.concatenate(pool_outputs, axis=-1, name='concatenate')
# pool_outputs = keras.layers.Flatten(data_format='channels_last', name='flatten')(pool_outputs)
# pool_outputs = keras.layers.Dropout(dropout_rate, name='dropout')(pool_outputs)


# pool_outputs = keras.layers.Flatten(data_format='channels_last', name='flatten')(embed)
# pool_outputs = keras.layers.Dropout(dropout_rate, name='dropout')(pool_outputs)


outputs = keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer='glorot_normal',
                             bias_initializer=keras.initializers.constant(0.1),
                             name='dense')(outputs)


model = keras.Model(inputs=[x1,x2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
functional_3 (Functional)       (None, 512, 128)     4369408     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 128)          0           functional_3[0][0]    

In [28]:
epochs = 10
batch_size = 2

history = model.fit([XX,XX_mask], YY, epochs=epochs, batch_size=batch_size,validation_split=0.3,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
# history = model.fit(X_train, Y_train, epochs=epochs,steps_per_epoch=100, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/10
25302/25302 [==============================] - 751s 30ms/step - loss: 0.8628 - accuracy: 0.7337 - val_loss: 0.8237 - val_accuracy: 0.7561
Epoch 2/10
25302/25302 [==============================] - 764s 30ms/step - loss: 0.6573 - accuracy: 0.7910 - val_loss: 0.8028 - val_accuracy: 0.7715
Epoch 3/10
25302/25302 [==============================] - 770s 30ms/step - loss: 0.6065 - accuracy: 0.8045 - val_loss: 0.7826 - val_accuracy: 0.7565
Epoch 4/10
25302/25302 [==============================] - 767s 30ms/step - loss: 0.5919 - accuracy: 0.8107 - val_loss: 0.6791 - val_accuracy: 0.7931
Epoch 5/10
25302/25302 [==============================] - 765s 30ms/step - loss: 0.5636 - accuracy: 0.8170 - val_loss: 0.7307 - val_accuracy: 0.7800
Epoch 6/10
25302/25302 [==============================] - 766s 30ms/step - loss: 0.5271 - accuracy: 0.8284 - val_loss: 0.6044 - val_accuracy: 0.8101
Epoch 7/10
25302/25302 [==============================] - 769s 30ms/step - loss: 0.4854 - accuracy: 0.8388

# texTCNN

In [26]:
from tensorflow.keras import layers

In [28]:
from tensorflow import keras

In [29]:
max_seq_length = 512
feature_size = max_seq_length
dropout_rate = 0.2
num_classes  = len( set( data['Query'] ) )


model_path = r"uncased_L-2_H-128_A-2/"
bert = load_trained_model_from_checkpoint(
    model_path + "bert_config.json",
    model_path + "bert_model.ckpt",
    seq_len = max_seq_length
)

for layer in bert.layers:
    layer.trainable = False
    
    
x1 = keras.Input(shape=(None,))
x2 = keras.Input(shape=(None,))
embed = bert([x1, x2])


x = layers.Conv1D(128, 5, activation="relu",kernel_initializer='glorot_normal',
                                   bias_initializer=keras.initializers.constant(0.1))(embed)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu",kernel_initializer='glorot_normal',
                                   bias_initializer=keras.initializers.constant(0.1))(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu",kernel_initializer='glorot_normal',
                                   bias_initializer=keras.initializers.constant(0.1))(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense( num_classes , activation="softmax")(x)
model = keras.Model(inputs=[x1,x2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# embed = keras.layers.Reshape((512, 128, 1), name='add_channel')(embed)



# pool_outputs = keras.layers.Flatten(data_format='channels_last', name='flatten')(embed)

# outputs = keras.layers.Dense(num_classes, activation='softmax',
#                              kernel_initializer='glorot_normal',
#                              bias_initializer=keras.initializers.constant(0.1),
#                              name='dense')(pool_outputs)


# model = keras.Model(inputs=[x1,x2], outputs=outputs)
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
functional_7 (Functional)       (None, 512, 128)     4369408     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 508, 128)     82048       functional_7[0][0]    

In [30]:
epochs = 2
batch_size = 32

history = model.fit([XX,XX_mask], YY, epochs=epochs, batch_size=batch_size,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
# history = model.fit(X_train, Y_train, epochs=epochs,steps_per_epoch=100, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/2


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node functional_9/conv1d/conv1d (defined at <ipython-input-30-c39918ba400f>:4) ]] [Op:__inference_train_function_9092]

Function call stack:
train_function


In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show()

In [ ]:
XX.shape

# random 500 data
# extract middle layer as document distribution presentation

In [ ]:
import random
import tensorflow as tf

x_list = []
y_list = []

index = 0
while True:
    if index >= 3000:
        break
    
    try:
        ranIndex = random.randint( 0,len( XX ) )

        layer_output = model.get_layer('concatenate').output
        intermediate_model = tf.keras.models.Model(inputs=model.input,outputs=layer_output)
        intermediate_prediction = intermediate_model.predict( XX[ranIndex:ranIndex+1] )

        Y_index = YY[ranIndex:ranIndex+1]

        # print( YY )
#         print( data['text_clean'][ranIndex] )
#         print()
#         print( data['Query'][ranIndex] )
#         print()

#         print( intermediate_prediction[0][0][0] )
        x_list.append( intermediate_prediction[0][0][0] )
#         y_list.append( data['Query'][ranIndex] )
        y_list.append( YY[ranIndex] )
#         print()
        index += 1

    except:
        print( 'Error' )

In [ ]:
c_list = [ np.argmax(y) for y in y_list ]

# t-sne visualization

In [ ]:
category_dict

In [ ]:
from sklearn.manifold import TSNE
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
 
X_tsne = TSNE(learning_rate=100).fit_transform( x_list )

fig = plt.figure(figsize=(20, 16))
ax = fig.add_subplot(111)

ind = 0

label_list = []
for tem in X_tsne:
    if c_list[ind] not in label_list:
        ax.plot( tem[0], tem[1],label = category_dict[c_list[ind]]  )
        label_list.append( c_list[ind] )
        
    ind +=1

plt.scatter( X_tsne[:, 0], X_tsne[:, 1], c=c_list )
plt.legend()